# 2x Single-Image Super-Resolution on Grayscale Images

**Assignment:** Individual Class Project<br>
**Author:** Richard Hemphill<br>
**ID:** 903877709<br>
**Class:** ECE5268 Theory of Neural Networks<br>
**Instructor:** Dr. Georgios C. Anagnostopoulos<br>
**Description:** Using small-sized grayscale images, construct a CNN-based architecture that will downscale (magnify) the images by a factor of 2.<br>
**Emphasis:** Describe the concept of single-image super-resolution, describe the architecture in sufficient detail and show indicative training and post-training results.<br>


**References:**
* https://www.kaggle.com/spaceengineer1/alexonly-greyscale
* https://www.kaggle.com/c/two-sigma-financial-news/discussion/83593
* https://scikit-image.org/docs/dev/auto_examples/transform/plot_rescale.html

In [196]:
# Imports
import os.path
from PIL import Image
from pathlib import Path
from zipfile import ZipFile
import tensorflow as tf
from tensorflow import keras
from kaggle.api.kaggle_api_extended import KaggleApi

In [197]:
# Constants
IMAGE_SET_OWNER = 'spaceengineer1'
IMAGE_SET_FILE = 'alexonly-greyscale'
ZIP_EXTENSION = 'zip'
IMAGE_EXTENSION = 'jpg'
PROCESSED_IMAGE_FOLDER ='dataSet'
TRAIN_FOLDER = 'train'
TEST_FOLDER = 'test'
CHANNELS = 1
ORIG_IMG_SIZE = 64
UPSCALE_FACTOR = 2
LOW_RES_IMG_SIZE = int(ORIG_IMG_SIZE/UPSCALE_FACTOR)

## Prepocessing

In [198]:
# Extract raw image set
def DownloadImageSet(imageSetOwner = IMAGE_SET_OWNER, imageSetFile = IMAGE_SET_FILE):
    zipFile = '{}.{}'.format(imageSetFile, ZIP_EXTENSION)
    if not os.path.isfile(zipFile):
        # connect to the Kaggle Database and download dataset
        api = KaggleApi()
        api.authenticate()
        api.dataset_download_files('{}/{}'.format(imageSetOwner, imageSetFile))
    # extract the dataset
    zf = ZipFile(zipFile)
    topDir = ''.join({item.split('/')[0] for item in zf.namelist()})
    if not os.path.isdir(topDir):
        zf.extractall() 
        zf.close()
        
    return topDir

In [199]:
# Convert images as needed for training
def PreProcessImages(rawFolder, processedFolder=PROCESSED_IMAGE_FOLDER, size=ORIG_IMG_SIZE, extension = IMAGE_EXTENSION):
    pathlist = Path(rawFolder).glob('**/*.{}'.format(extension))
    for rawImg in pathlist:
        newImg = str(rawImg).replace(rawFolder, processedFolder)
        if os.path.isfile(newImg):
            continue
        img = Image.open(rawImg)
        resizeImg = img.resize((size,size), resample=Image.ANTIALIAS)
        os.makedirs(os.path.dirname(newImg), exist_ok=True)
        resizeImg.save(newImg)

    trainPath = '{}/{}'.format(processedFolder, TRAIN_FOLDER)
    testPath = '{}/{}'.format(processedFolder, TEST_FOLDER)
    return trainPath, testPath

In [200]:
# Pre Process Images
imgFolder = DownloadImageSet()
trainPath, testPath = PreProcessImages(rawFolder=imgFolder)

## Create Model

In [201]:
def SuperResolution():
    model = keras.models.Sequential()
    model.add(keras.layers.Conv2D(filters=CHANNELS * LOW_RES_IMG_SIZE^2, kernel_size=9, activation='relu', padding='same'))
    model.add(keras.layers.Conv2D(filters=32, kernel_size=1, activation='relu', padding='same'))
    model.add(keras.layers.Conv2D(filters=CHANNELS * ORIG_IMG_SIZE^2, kernel_size=5, activation='relu', padding='same'))
    return model

In [202]:
sr = SuperResolution()